In [ ]:
import http.client
import json
import pandas as pd
import re

conn = http.client.HTTPSConnection("instagram-scraper-stable-api.p.rapidapi.com")
API_HOST = "instagram-scraper-stable-api.p.rapidapi.com"
API_KEY = "ecd5fe584amshcc4f497af92494ap187424jsn4d00070a199d"

In [ ]:
# API request function
def fetch_instagram_data(endpoint, payload=None, method="GET"):
    conn = http.client.HTTPSConnection(API_HOST)
    headers = {
        'x-rapidapi-key': API_KEY,
        'x-rapidapi-host': API_HOST,
        'Content-Type': "application/x-www-form-urlencoded"
    }

    if method == "GET":
        conn.request("GET", endpoint, headers=headers)
    else:
        conn.request("POST", endpoint, body=payload, headers=headers)

    response = conn.getresponse()
    data = response.read().decode("utf-8", errors="ignore")
    conn.close()

    return json.loads(data)

# Function to search Instagram users
def search_users(query):
    endpoint = "/search_ig.php"
    payload = f"search_query={query}"
    data = fetch_instagram_data(endpoint, payload, method="POST")

    users_data = []
    for item in data.get("users", []):
        user = item.get("user", {})
        users_data.append({
            "username": user.get("username"),
            "full_name": user.get("full_name"),
            "is_verified": user.get("is_verified", False),
            "follower_count": user.get("search_social_context", "0"),
            "profile_pic_url": user.get("profile_pic_url"),
            "user_id": user.get("id")
        })

    return pd.DataFrame(users_data)

# Function to search Instagram hashtags
def search_hashtags(hashtag):
    endpoint = f"/search_hashtag.php?hashtag={hashtag}"
    data = fetch_instagram_data(endpoint)

    hashtags_data = []
    for tag in data.get("hashtags", []):
        hashtags_data.append({
            "hashtag": tag.get("name"),
            "post_count": tag.get("search_social_context", "0"),
            "id": tag.get("id")
        })

    return pd.DataFrame(hashtags_data)

# Function to fetch post engagement data
def get_post_engagement(post_id):
    likes_endpoint = f"/get_post_likers.php?post_id={post_id}"
    comments_endpoint = f"/get_post_comments.php?post_id={post_id}&sort_order=popular"

    likes_data = fetch_instagram_data(likes_endpoint)
    comments_data = fetch_instagram_data(comments_endpoint)

    return {
        "post_id": post_id,
        "likes_count": len(likes_data.get("users", [])),
        "comments_count": len(comments_data.get("comments", []))
    }

# Example usage
query = "fortnite"
hashtag_query = "gaming"

# Search users
users_df = search_users(query)
print("User Search Results:")
print(users_df)

# Search hashtags
hashtags_df = search_hashtags(hashtag_query)
print("\nHashtag Search Results:")
print(hashtags_df)

# Fetch engagement for first post (if any users have posts)
if not users_df.empty:
    post_id_example = users_df.iloc[0]["user_id"]
    engagement_data = get_post_engagement(post_id_example)
    print("\nPost Engagement Data:")
    print(engagement_data)

User Search Results:
Empty DataFrame
Columns: []
Index: []

Hashtag Search Results:
Empty DataFrame
Columns: []
Index: []


In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import re

# Function to convert "K"/"M" follower counts to numbers
def convert_follower_count(follower_str):
    if isinstance(follower_str, str):
        # Extract numeric part using regex
        match = re.search(r"(\d+(?:\.\d+)?)([MK]?)", follower_str)
        if match:
            num = float(match.group(1))
            suffix = match.group(2)
            if suffix == "M":
                return num * 1e6
            elif suffix == "K":
                return num * 1e3
            else:
                return num  # Handle cases like "27 followers"
    return float(follower_str) if follower_str else 0

# Function to preprocess user data
def preprocess_users(df):
    if df.empty:
        return df

    df["follower_count"] = df["follower_count"].apply(convert_follower_count)
    df["is_verified"] = df["is_verified"].astype(int)  # Convert True/False to 1/0

    # Normalize numerical columns
    scaler = MinMaxScaler()
    df[["follower_count"]] = scaler.fit_transform(df[["follower_count"]])

    return df

def preprocess_hashtags(df):
    if df.empty:
        return df  # <- Change this line

    df["post_count"] = df["post_count"].apply(lambda x: int(re.sub(r"[^\d]", "", str(x))) if str(x).isdigit() else 0)

    scaler = MinMaxScaler()
    if not df[["post_count"]].empty:  # Ensure it has valid numeric data
        df[["post_count"]] = scaler.fit_transform(df[["post_count"]])

    return df

# Function to preprocess post engagement data
def preprocess_engagement(df):
    if df.empty:
        return df

    scaler = MinMaxScaler()
    df[["likes_count", "comments_count"]] = scaler.fit_transform(df[["likes_count", "comments_count"]])

    return df

# Apply preprocessing
users_df = preprocess_users(users_df)
hashtags_df = preprocess_hashtags(hashtags_df)

print("Preprocessed User Data:")
print(users_df.head())

print("\nPreprocessed Hashtag Data:")
print(hashtags_df.head())

Preprocessed User Data:
Empty DataFrame
Columns: []
Index: []

Preprocessed Hashtag Data:
Empty DataFrame
Columns: []
Index: []


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Sample engagement data (Replace this with actual API data)
engagement_data = pd.DataFrame({
    "user_id": ["1192899491", "3015537629", "70940743993", "26175663062"],
    "likes_count": [1200, 450, 2300, 3400],
    "comments_count": [50, 30, 100, 120]
})

# Normalize numerical features
scaler = MinMaxScaler()

# Process User Data (Already preprocessed in Step 2)
users_df["follower_count"] = scaler.fit_transform(users_df[["follower_count"]])

# Process Engagement Data
if not engagement_data.empty:
    engagement_data[["likes_count", "comments_count"]] = scaler.fit_transform(
        engagement_data[["likes_count", "comments_count"]]
    )

# Merge user data with engagement data (on user_id)
df_combined = users_df.merge(engagement_data, on="user_id", how="left")

# Fill missing values with 0 (for users without engagement data)
df_combined = df_combined.fillna(0)

# Display final feature matrix
print("Feature Matrix for Two-Tower Model:")
print(df_combined.head())

KeyError: "None of [Index(['follower_count'], dtype='object')] are in the [columns]"

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model

# Define input shapes based on feature matrix
user_input = Input(shape=(2,), name="user_features")  # [is_verified, follower_count]
content_input = Input(shape=(2,), name="content_features")  # [likes_count, comments_count]

# User Tower
user_dense = Dense(64, activation='relu')(user_input)
user_dense = Dense(32, activation='relu')(user_dense)
user_embedding = Dense(16, activation='relu', name="user_embedding")(user_dense)

# Content Tower
content_dense = Dense(64, activation='relu')(content_input)
content_dense = Dense(32, activation='relu')(content_dense)
content_embedding = Dense(16, activation='relu', name="content_embedding")(content_dense)

# Compute similarity (dot product)
similarity = tf.keras.layers.Dot(axes=1, normalize=True, name="similarity")([user_embedding, content_embedding])

# Define Model
model = Model(inputs=[user_input, content_input], outputs=similarity)
model.compile(optimizer='adam', loss='mse')  # Using MSE for ranking

# Summary of Model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user_features             │ (None, 2)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ content_features          │ (None, 2)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │            192 │ user_features[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 64)             │            192 │ content_features[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 32)             │          2,080 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 32)             │          2,080 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ user_embedding (Dense)    │ (None, 16)             │            528 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ content_embedding (Dense) │ (None, 16)             │            528 │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ similarity (Dot)          │ (None, 1)              │              0 │ user_embedding[0][0],  │
│                           │                        │                │ content_embedding[0][… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 5,600 (21.88 KB)

 Trainable params: 5,600 (21.88 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Convert DataFrames to NumPy arrays
user_features_np = users_df[["follower_count", "is_verified"]].values
# Get 'likes_count', 'comments_count' from df_combined created in the previous cell
content_features_np = df_combined[["likes_count", "comments_count"]].values

labels = np.random.randint(0, 2, size=len(users_df))  # Replace with actual labels

# Split into Train and Validation sets
X_train_user, X_val_user, X_train_content, X_val_content, y_train, y_val = train_test_split(
    user_features_np, content_features_np, labels, test_size=0.2, random_state=42
)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(
    [X_train_user, X_train_content], y_train,
    validation_data=([X_val_user, X_val_content], y_val),
    epochs=32, batch_size=8
)

Epoch 1/32
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5000 - loss: 1.1847 - val_accuracy: 1.0000 - val_loss: 0.0770
Epoch 2/32
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 382ms/step - accuracy: 0.5000 - loss: 1.0235 - val_accuracy: 1.0000 - val_loss: 0.1251
Epoch 3/32
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.5000 - loss: 0.9034 - val_accuracy: 1.0000 - val_loss: 0.1817
Epoch 4/32
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.5000 - loss: 0.8133 - val_accuracy: 1.0000 - val_loss: 0.2450
Epoch 5/32
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 0.5000 - loss: 0.7436 - val_accuracy: 1.0000 - val_loss: 0.2979
Epoch 6/32
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.5000 - loss: 0.6982 - val_accuracy: 1.0000 - val_loss: 0.3827
Epoch 7/32
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.5000 - loss: 0.6434 - val_accuracy: 1.0000 - val_loss: 0.4510
Epoch 8/32
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 0.5000 - loss: 0.6073 - val_accuracy: 1.0000 - val_loss: 0.531

In [ ]:
# Save the model
model.save("two_tower_recommendation_model.h5")
print("✅ Model training complete!")

✅ Model training complete!


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate([user_features_np, content_features_np], labels, verbose=1)

print(f"Model Loss: {loss:.4f}")
print(f"Model Accuracy: {accuracy:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.8000 - loss: 3.4749
Model Loss: 3.4749
Model Accuracy: 0.8000


In [ ]:
import numpy as np
import pandas as pd

# Generate similarity scores using the trained model
predictions = model.predict([user_features_np, content_features_np])

# Add predictions to the DataFrame
users_df["similarity_score"] = predictions

# Sort results by similarity score (higher is better)
top_results = users_df.sort_values(by="similarity_score", ascending=False).head(5)

# Display the top 5 most relevant profiles/posts
print("Top 5 Most Relevant Results:")
print(top_results[["username", "full_name", "similarity_score"]])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
Top 5 Most Relevant Results:
           username              full_name  similarity_score
0          fortnite               Fortnite          0.999738
3        bestxpmaps  Best Fortnite XP Maps          0.508192
2  fortniteskin_sus       FORTNITE SKIN 🍌🍑          0.439755
1         forrtnite          Fortnite News          0.000000
4         fnshopbot     Fortnite Item Shop          0.000000
